In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [7]:
# Problem parameters
# (Double-check with Ragusa on appropriate parameters)
sigma_s = 0
sigma_t = 1
q_ext = 3
q = q_ext/2.0
# Vacuum boundary conditions
psi_left = 0
psi_right = 0

# Number of points in domain
npts = 21
# Left- and right-endpoints of domain
a = 0
b = 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Quadrature order (needs to be odd to avoid mu = 0)
n = 16
mu_s, weights = scipy.special.roots_legendre(n)
# By default, mu_s entries go from negative to positive, and we want the opposite
mu_s = np.flip(mu_s)

# Array to store angular flux values (each row corresponds to a different angle,
# and there as many columns as there are points)
psi_vals = np.zeros((mu_s.size, npts))
# Impose BCs
psi_vals[0:int((n/2)), 0]  = psi_left # Impose left BC for forward sweep
psi_vals[int((n/2)):, -1] = psi_right # Impose right BC for reverse sweep

for mu_idx, mu_d in enumerate(mu_s):
    # If angle is greater than 0, perform a forward sweep (left -> right)
    if mu_d > 0:
        # Loop through points (cell boundaries)
        for pt_idx in np.arange(npts):
            # print(f"Forward sweep: {cell_idx}")
            # Point to the right of current point       
            if pt_idx < (npts - 1):
                pt_r_idx = pt_idx + 1 
                psi_vals[mu_idx, pt_r_idx] = (q*dx + mu_d*psi_vals[mu_idx, pt_idx] - (sigma_t/2.0)*psi_vals[mu_idx, pt_idx]*dx) / \
                                            (mu_d + ((sigma_t * dx)/2.0))
    # If angle is less than 0, perform a reverse sweep (right -> left)            
    else:
        for pt_idx in np.arange(npts-1, -1, -1):
                # Reverse sweep    
                pt_l_idx = pt_idx - 1
                psi_vals[mu_idx, pt_l_idx] = (q*dx - mu_d*psi_vals[mu_idx, pt_idx] - (sigma_t/2.0)*psi_vals[mu_idx, pt_idx]*dx) / \
                                            (-mu_d + ((sigma_t * dx)/2.0))

In [8]:
# Compute average flux (average of each consecutive 2 angular flux values for each angle)
average_flux = psi_vals[:, 0:(npts-1)] + psi_vals[:, 1:npts]/2.0
# print(average_flux[0:int(n/2), :])
print(psi_vals[int(n/2):, :])

[[0.03696763 0.14604839 0.24975253 0.34834508 0.44207798 0.53119078
  0.61591118 0.69645571 0.77303018 0.84583028 0.91504205 0.98084236
  1.04339937 1.10287292 1.15941502 1.21317014 1.26427567 1.31286219
  1.35905388 1.40296877]
 [0.03867674 0.15271244 0.26086743 0.36344497 0.4607327  0.5530034
  0.64051579 0.72351526 0.80223453 0.87689433 0.94770399 1.01486208
  1.07855689 1.13896702 1.19626186 1.25060207 1.30214    1.35102018
  1.39737966 1.44134843]
 [0.04210497 0.16605611 0.28304863 0.39347321 0.49769855 0.59607268
  0.68892409 0.77656283 0.85928153 0.93735641 1.01104817 1.08060288
  1.14625279 1.20821711 1.26670276 1.32190503 1.37400824 1.42318638
  1.46960367 1.51341509]
 [0.04805201 0.18912938 0.32116801 0.44474701 0.56040839 0.66865942
  0.76997488 0.86479912 0.95354803 1.03661086 1.11435191 1.18711214
  1.25521067 1.31894617 1.37859818 1.43442833 1.48668147 1.53558679
  1.58135878 1.62419818]
 [0.0583316  0.22878963 0.38599021 0.53096444 0.66466324 0.78796354
  0.90167411 1.00